#### FILTER BİTTİKTEN SONRA BAŞLANACAK (Yazılıyor)

In [21]:
import librosa
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import warnings
warnings.filterwarnings("ignore")

In [42]:
audio, sr = librosa.load("test.mp3", sr=16000)
audio = librosa.feature.mfcc(audio, sr=sr)
audio = librosa.power_to_db(audio, ref=np.max)
audio = torch.from_numpy(audio.T)
audio = audio.unsqueeze(0)
audio.shape

torch.Size([1, 125, 128])

In [43]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        
        out,hx = self.rnn(x, h0)
        out = self.fc(out[:,-1,:])
        out = self.sigmoid(out)
        return out[0]

In [44]:
model = RNN(input_size=20,hidden_size=256,output_size=1)

In [45]:
model.train()

RNN(
  (rnn): RNN(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [46]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [47]:
label = torch.tensor([1], dtype=torch.float)

In [48]:
noise,sr = librosa.load("Datasets/Filter_Datasets/noise_train/AirConditioner_1.wav",sr=16000)
noise = librosa.feature.mfcc(noise, sr=sr)
noise = librosa.power_to_db(noise, ref=np.max)
noise = torch.from_numpy(noise.T)
noise = noise.unsqueeze(0)

In [49]:
label_noise = torch.tensor([0], dtype=torch.float)

In [50]:
data_List = []
data_List.append([noise,label_noise])
data_List.append([audio,label])

In [51]:
for epoch in range(100):
    for x,y in data_List:
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch: {epoch}, Loss: {loss.item():.4f}")

Epoch: 0, Loss: 0.5960
Epoch: 0, Loss: 2.1689
Epoch: 10, Loss: 0.7897
Epoch: 10, Loss: 0.6005
Epoch: 20, Loss: 0.7348
Epoch: 20, Loss: 0.6649
Epoch: 30, Loss: 0.6814
Epoch: 30, Loss: 0.7118
Epoch: 40, Loss: 0.6960
Epoch: 40, Loss: 0.6963
Epoch: 50, Loss: 0.6985
Epoch: 50, Loss: 0.6947
Epoch: 60, Loss: 0.6959
Epoch: 60, Loss: 0.6972
Epoch: 70, Loss: 0.6964
Epoch: 70, Loss: 0.6966
Epoch: 80, Loss: 0.6966
Epoch: 80, Loss: 0.6964
Epoch: 90, Loss: 0.6965
Epoch: 90, Loss: 0.6965


In [32]:
model(audio)

tensor([0.9982], grad_fn=<SelectBackward0>)

In [33]:
model(noise)

tensor([0.0024], grad_fn=<SelectBackward0>)

In [34]:
audio,sr = librosa.load("Datasets/Filter_Datasets/clean_train/p234_113.wav",sr=16000)
audio = librosa.feature.mfcc(audio, sr=sr)
audio = librosa.power_to_db(audio, ref=np.max)
audio = torch.from_numpy(audio.T)
audio = audio.unsqueeze(0)

In [35]:
model(audio)

tensor([0.7774], grad_fn=<SelectBackward0>)

In [36]:
audio,sr = librosa.load("Datasets/Filter_Datasets/noise_train/Washing_1.wav",sr=16000)
audio = librosa.feature.mfcc(audio, sr=sr)
audio = librosa.power_to_db(audio, ref=np.max)
audio = torch.from_numpy(audio.T)
audio = audio.unsqueeze(0)

In [37]:
model(audio)

tensor([0.7279], grad_fn=<SelectBackward0>)

In [40]:
audio,sr = librosa.load("Kayıt.wav",sr=16000)
audio = librosa.feature.mfcc(audio, sr=sr)
audio = librosa.power_to_db(audio, ref=np.max)
audio = torch.from_numpy(audio.T)
audio = audio.unsqueeze(0)

In [41]:
model(audio)

tensor([0.6898], grad_fn=<SelectBackward0>)